In [92]:
from bs4 import BeautifulSoup
from urllib.request import Request,urlopen

req = Request('https://www.jumia.co.ke/smartphones/', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_soup = BeautifulSoup(webpage, 'html.parser')
containers = page_soup.findAll("a", {"class":"core"})

f = open("smartphones.csv", "w")
headers = "description,new_price,old_price,discount,ratings\n"
f.write(headers)
for container in containers:
    container_description  = container.findAll("div",{"class":"info"})
    description = container_description[0].h3.text
    new_price = container_description[0].findAll("div",{"class":"prc"})[0].text
    if container_description[0].findAll("div", {"class":"s-prc-w"}):
        old_price = container_description[0].findAll("div", {"class":"s-prc-w"})[0].find("div",{"class":"old"}).text
        discount = container_description[0].findAll("div", {"class":"s-prc-w"})[0].find("div",{"class":"tag _dsct _sm"}).text
    if container_description[0].findAll("div",{"class":"rev"}):
        ratings = container_description[0].findAll("div",{"class":"rev"})[0].find("div",{"class":"stars _s"}).text
    f.write(description.replace(",","|") + "," + new_price + "," + old_price + "," + discount + "," + ratings + "\n")
f.close()
    

In [27]:
import pandas as pd
j = pd.read_csv('Records.csv')
j.columns = j.columns.str.lower()
j.columns = j.columns.str.replace(" ","_")
# b = pd.DataFrame(j)
# print(b.info())
# print(b[b['Country']=="Canada"])
# print(j)

#### Saving data after transformation to another csv file

In [73]:
j.to_csv("records2.csv", index=False)
print(j.info())
# print("success!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   region          1000 non-null   object 
 1   country         1000 non-null   object 
 2   item_type       1000 non-null   object 
 3   sales_channel   1000 non-null   object 
 4   order_priority  1000 non-null   object 
 5   order_date      1000 non-null   object 
 6   order_id        1000 non-null   int64  
 7   ship_date       1000 non-null   object 
 8   units_sold      1000 non-null   int64  
 9   unit_price      1000 non-null   float64
 10  unit_cost       1000 non-null   float64
 11  total_revenue   1000 non-null   float64
 12  total_cost      1000 non-null   float64
 13  total_profit    1000 non-null   float64
dtypes: float64(5), int64(2), object(7)
memory usage: 109.5+ KB
None


#### Connecting to database and creating table

In [76]:
import pymysql
db = pymysql.connect("localhost","root","makobongo","pyDatabase" )
cursor = db.cursor()
sql = """CREATE TABLE records (
        region VARCHAR(255),
        country VARCHAR(255),
        item_type VARCHAR(255),
        sales_channel VARCHAR(255),
        order_priority char,
        order_date VARCHAR(255),
        order_id int,
        ship_date VARCHAR(255),
        units_sold int,
        unit_price float,
        unit_cost float,
        total_revenue float,
        total_cost float,
        total_profit float
        )"""
try:
    cursor.execute(sql)
    print("success!")
except:
    print("fail!")

success!


#### Reading from CSV and inserting into csv records to mysql database

In [77]:
import csv
with open('records2.csv') as csvfile:
    sReader = csv.reader(csvfile)
    next(sReader)
    for row in sReader:
#         print(row[0])
        sql = """
            INSERT INTO records(
                region,
                country,
                item_type,
                sales_channel,
                order_priority,
                order_date,
                order_id,
                ship_date,
                units_sold,
                unit_price,
                unit_cost,
                total_revenue,
                total_cost,
                total_profit
            ) VALUES (
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s,
                %s
            )
        """
        val = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13])
        try:
            cursor.execute(sql, val)
            db.commit()
        except(e):
            print(e)